# Multi Fabric - Commissioning and Interactions

<a href="http://35.236.121.59/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fproject-chip%2Fconnectedhomeip&urlpath=lab%2Ftree%2Fconnectedhomeip%2Fdocs%2Fguides%2Frepl%2FMatter%2520-%2520Multi%2520Fabric%2520Commissioning.ipynb&branch=master">
<img src="https://i.ibb.co/hR3yWsC/launch-playground.png" alt="drawing" width="130"/>
</a>
<br></br>

This walks through creating multiple controllers on multiple fabrics, using those controllers to commission a target onto those fabrics and finally, interacting with them using the interaction model.

## CertificateAuthority, FabricAdmins and Controllers

The `CertificateAuthorityManager` class manages a set of `CertificateAuthority` instances (both present in `chip.CertificateAuthority` package). a `CertificateAuthority` represents an operational root of trust, a Root Certificate Authority (CA) with a root key pair with the associated public key (i.e "Root PK"). The `CertificateAuthority` class manages a list of `FabricAdmins` adminstering a fabric within that CA.

The `FabricAdmin` class (present in the `chip.FabricAdmin` package) is responsible for administering a fabric. It houses the Fabric ID and is managed by a `CertificateAuthority`.

The `FabricAdmin` can be used to vend `ChipDeviceController` objects that represent a controller instance with a specific identity grounded in the admin's fabric. This controller can then be used to commission and interact with devices.

## Clear Persisted Storage

Let's clear out our persisted storage (if one exists) to start from a clean slate.

In [1]:
import os, subprocess

if os.path.isfile('/tmp/repl-storage.json'):
    os.remove('/tmp/repl-storage.json')

# So that the all-clusters-app won't boot with stale prior state.    
os.system('rm -rf /tmp/chip_*')

0

## Initialization

Let's first begin setting up by importing some key modules that are needed to make it easier for us to interact with the Matter stack.

`ChipReplStartup.py` is run within the global namespace. This results in all of its imports being made available here.

> **NOTE**: _This is not needed if you launch the REPL from the command-line._

In [2]:
%reset -f
import importlib.util
spec = importlib.util.find_spec('chip.ChipReplStartup')
%run {spec.origin} --debug

 Replacing  store path ./credentials/development/paa-root-certs with 
/home/root/repos/connectedhomeip/credentials/development/paa-root-certs
Note that you are still running from 
/home/root/repos/connectedhomeip/docs/development_controllers/chip-repl so other relative
paths may be off.

[1738152867.148313][2264454:2264454] CHIP:CTL: Setting attestation nonce to random value
[1738152867.148534][2264454:2264454] CHIP:CTL: Setting CSR nonce to random value
[1738152867.160741][2264454:2264454] CHIP:DL: NVS set: chip-factory/unique-id = "3844ABE883465BB4"
[1738152867.161683][2264454:2264454] CHIP:DL: NVS set: chip-factory/vendor-id = 65521 (0xFFF1)
[1738152867.162893][2264454:2264454] CHIP:DL: NVS set: chip-factory/product-id = 32769 (0x8001)
[1738152867.164421][2264454:2264454] CHIP:DL: NVS set: chip-counters/reboot-count = 1 (0x1)
[1738152867.165934][2264454:2264454] CHIP:DL: NVS set: chip-counters/total-operational-hours = 0 (0x0)
[1738152867.167463][2264454:2264454] CHIP:DL: NVS set: chip-counters/boot-reason = 0 (0x0)
[1738152867.168925][2264454:2264454] CHIP:DL: NVS set: chip-config/regulatory-location = 0 (0x0)
[1738152867.170369][2264454:2264454] CHIP:DL: NVS set: chip-config/location-capability = 2 (0x2)
[1738152867.170831][2264454:2264454] CHIP:DL: Got Ethernet i

─────────────────────────────────────────────────── Matter REPL ───────────────────────────────────────────────────

            Welcome to the Matter Python REPL!

            For help, please type matterhelp()

            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.

            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-01-29 13:14:27 ThinkPad chip.storage[2264454] ERROR [Errno 2] No such file or directory: '/tmp/repl-storage.json'
2025-01-29 13:14:27 ThinkPad chip.storage[2264454] CRITICAL Could not load configuration from /tmp/repl-storage.json - resetting configuration...
2025-01-29 13:14:27 ThinkPad chip.storage[2264454] WARNING No valid SDK configuration present - clearing out configuration
2025-01-29 13:14:27 ThinkPad chip.storage[2264454] WARNING No valid REPL configuration present - clearing out configuration


The following objects have been created:

certificateAuthorityManager:    Manages a list of CertificateAuthority instances.
        caList:                         The list of CertificateAuthority instances.
        caList[n].adminList[m]:         A specific FabricAdmin object at index m for the nth CertificateAuthority 
instance.

Default CHIP Device Controller (NodeId: 112233): has been initialized to manage caList[0].adminList[0] (FabricId = 
1), and is available as devCtrl

At startup, the `certificateAuthorityManager` within REPL will attempt to find `CertificateAuthority` instances present in persistent storage.If it can't find any (as is the case here), it will create a new `CertificateAuthority` instance at CA index 0. This `CertificateAuthority` instance will construct a default `FabricAdmin` object on FabricAdmin index 0 with a Fabric ID = 1.

In [3]:
inspect(caList[0].adminList[0])

╭───────────────────────────── <class 'chip.FabricAdmin.FabricAdmin'> ─────────────────────────────╮
│ Administers a fabric associated with a unique FabricID under a given CertificateAuthority        │
│ instance.                                                                                        │
│                                                                                                  │
│ ╭──────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <chip.FabricAdmin.FabricAdmin object at 0x74c3436d78e0>                                      │ │
│ ╰──────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                  │
│              caIndex = 1                                                                         │
│ certificateAuthority = <chip.CertificateAuthority.CertificateAuthority object at 0x74c32c8ba680> │
│             fabricId = 1                                                                         │
│             vendorId = 65521                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

The Newly created `FabricAdmin` will automatically construct a device controller (`devCtrl`) on it's own fabric. 

In [4]:
inspect(devCtrl)

╭────────────────── <class 'chip.ChipDeviceCtrl.ChipDeviceController'> ───────────────────╮
│ The ChipDeviceCommissioner binding, named as ChipDeviceController                       │
│                                                                                         │
│ ╭─────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ <chip.ChipDeviceCtrl.ChipDeviceController object at 0x74c32c8baa10>                 │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                         │
│      activeList = {<chip.ChipDeviceCtrl.ChipDeviceController object at 0x74c32c8baa10>} │
│         caIndex = 1                                                                     │
│         devCtrl = c_void_p(128380733235904)                                             │
│     fabricAdmin = <chip.FabricAdmin.FabricAdmin object at 0x74c3436d78e0>               │
│        fabricId = 1                                                                     │
│        isActive = True                                                                  │
│            name = 'caIndex(1)/fabricId(0x0000000000000001)/nodeId(0x000000000001B669)'  │
│          nodeId = 112233                                                                │
│ pairingDelegate = c_void_p(128380733235776)                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────╯

### Commission onto Fabric 1

#### Launch Server

Let's launch an instance of the `chip-all-clusters-app`.

In [5]:
import time, os
import subprocess
os.system('pkill -f chip-all-clusters-app')
time.sleep(1)

CI_APP_PATH = '../../../out/linux-x64-all-clusters-ipv6only-no-ble-no-wifi-clang-test/chip-all-clusters-app'
LOCAL_APP_PATH = '../../../out/linux-x64-all-clusters/chip-all-clusters-app'

# Check if the app built by GitHub Actions is present. Otherwise use the standard all-clusters-app 
if (os.path.isfile(CI_APP_PATH)):
    appPath = CI_APP_PATH
else:
    appPath = LOCAL_APP_PATH

process = subprocess.Popen(appPath, stdout=subprocess.DEVNULL)
time.sleep(1)

#### Commission Target

Commission the target onto Fabric 1 using the default device controller instance with a NodeId of 2.

In [6]:
await devCtrl.CommissionOnNetwork(2, 20202021)

2

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 1.

In [7]:
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 4246564825,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04\xffKd\xd3v\xa97c\x87\r\xadXF\xd2?\n\x08\x1f\rZE\x91\x0c_\x1e`"\xfbwZ\x9c\xaf!\x10=\xd4\x03\xf5[}\xb3\x08\xa2\xe5\x04\x7f\x06\x1c\nEa\x02q\xeeK\x91\xda\x1e\xd2\\i\xef\x05#',
│   │   │   │   │   vendorID=65521,
│   │   │   │   │   fabricID=1,
│   │   │   │   │   nodeID=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=1
│   │   │   │   )
│   │   │   ]
│   │   }
│   }
}

### Commission onto Fabric 2

#### Create a new CertificateAuthority and a new FabricAdmin
We create a new `CertificateAuthority` instance, and then create a new `FabricAdmin` that administers Fabric 2 within that CA.

In [8]:
certificateAuthority = certificateAuthorityManager.NewCertificateAuthority()

fabricAdmin2 = certificateAuthority.NewFabricAdmin(vendorId=0xFFF1, fabricId=2)

Two different `CertificateAuthority` instances are now present

In [9]:
caList


[
│   <chip.CertificateAuthority.CertificateAuthority object at 0x74c32c8ba680>,
│   <chip.CertificateAuthority.CertificateAuthority object at 0x74c32c8b9c60>
]

Here's a brief peek at the JSON data that is in the persisted storage file.

In [10]:
builtins.chipStack.GetStorageManager().jsonData


{
│   'sdk-config': {
│   │   'g/lkgt': 'FSYAgKi8LBg=',
│   │   'g/gcc': '2rpTAQ==',
│   │   'g/gdc': 'bNE5Bw==',
│   │   'ExampleOpCredsCAKey1': 'BP9LZNN2qTdjhw2tWEbSPwoIHw1aRZEMXx5gIvt3WpyvIRA91AP1W32zCKLlBH8GHApFYQJx7kuR2h7SXGnvBSMAh4AKfr1HHSs7hYln6Oif+DTdrYIartnC0MufgIwjOw==',
│   │   'ExampleOpCredsICAKey1': 'BAOZBNHF54zmSMKesumS33XPPiYkNuFD4EneHIbBlGw5R74leYynlviYRo6pJJgsBFhZpPSysbanxd6FGhDMSSvkfszhKxNneEFnLM3zg/NWzKzqYglnZf4cjVFDVoKN1A==',
│   │   'ExampleCARootCert1': 'MIIBljCCATygAwIBAgIBATAKBggqhkjOPQQDAjAiMSAwHgYKKwYBBAGConwBBAwQMDAwMDAwMDAwMDAwMDAwMTAeFw0yMTAxMDEwMDAwMDBaFw0zMDEyMzAwMDAwMDBaMCIxIDAeBgorBgEEAYKifAEEDBAwMDAwMDAwMDAwMDAwMDAxMFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAE/0tk03apN2OHDa1YRtI/CggfDVpFkQxfHmAi+3danK8hED3UA/VbfbMIouUEfwYcCkVhAnHuS5HaHtJcae8FI6NjMGEwDwYDVR0TAQH/BAUwAwEB/zAOBgNVHQ8BAf8EBAMCAQYwHQYDVR0OBBYEFK4R1kjK4X+tUcRsjhHsVt60AN7yMB8GA1UdIwQYMBaAFK4R1kjK4X+tUcRsjhHsVt60AN7yMAoGCCqGSM49BAMCA0gAMEUCIQD07YMcVJ+u1B9LehEK3qnyXronDsmovVX/qXO34/JgbgIgEsIzOThDOapZJ

#### Create a new ChipDeviceController on Fabric 2

In [11]:
devCtrl2 = fabricAdmin2.NewController()

#### Open Commissioning Window

The Controller of Fabric 1 must open the commissioning window that allows the Commissioner of Fabric 2 to join the node to its fabric.

In [12]:
await devCtrl.SendCommand(2, 0, Clusters.AdministratorCommissioning.Commands.OpenBasicCommissioningWindow(180),  timedRequestTimeoutMs=10000)

In [13]:
await devCtrl2.CommissionOnNetwork(2, 20202021)

2

### Read OpCreds Cluster

Read out the OpCreds cluster to confirm membership into Fabric 2.

In [14]:
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 4246564829,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04\xffKd\xd3v\xa97c\x87\r\xadXF\xd2?\n\x08\x1f\rZE\x91\x0c_\x1e`"\xfbwZ\x9c\xaf!\x10=\xd4\x03\xf5[}\xb3\x08\xa2\xe5\x04\x7f\x06\x1c\nEa\x02q\xeeK\x91\xda\x1e\xd2\\i\xef\x05#',
│   │   │   │   │   vendorID=65521,
│   │   │   │   │   fabricID=1,
│   │   │   │   │   nodeID=2,
│   │   │   │   │   label='',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04pKSE\xb6\x07\x04\n\xa5p\xbb\x1f\xa0Z\xb8HS\xaa\xaa\xe9|\xf1\xaf+\x0e\xa6] 7\x8f\x10\xe5\xa6\xb6\x15$k\x08#X\xa5\x8f+\xd3M\x99\x06\xc1\x86\x8e\xda\xd3{&\xaf)\x81\xa1h\xc7$\xc0\xad\x9b',
│   │   │   │   │   vendorID=65521,
│   │   │   │   │   fabric

## Relaunch REPL

Let's simulate re-launching the REPL to show-case the capabilities of the persistence storage and its mechanics.

In [15]:
%reset -f
import importlib.util
spec = importlib.util.find_spec('chip.ChipReplStartup')
%run {spec.origin}

 Replacing  store path ./credentials/development/paa-root-certs with 
/home/root/repos/connectedhomeip/credentials/development/paa-root-certs
Note that you are still running from 
/home/root/repos/connectedhomeip/docs/development_controllers/chip-repl so other relative
paths may be off.

─────────────────────────────────────────────────── Matter REPL ───────────────────────────────────────────────────

            Welcome to the Matter Python REPL!

            For help, please type matterhelp()

            To get more information on a particular object/class, you can pass
            that into matterhelp() as well.

            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

The following objects have been created:

certificateAuthorityManager:    Manages a list of CertificateAuthority instances.
        caList:                         The list of CertificateAuthority instances.
        caList[n].adminList[m]:         A specific FabricAdmin object at index m for the nth CertificateAuthority 
instance.

Default CHIP Device Controller (NodeId: 112233): has been initialized to manage caList[0].adminList[0] (FabricId = 
1), and is available as devCtrl

The REPL now loaded (through the `certificateAuthorityManager`) the two `certificateAuthority` instances that were created in the previous session into the `CaList`. It has also created a default controller on the first fabric in that list (Fabric 1) as `devCtrl`.

In [16]:
caList


[
│   <chip.CertificateAuthority.CertificateAuthority object at 0x74c32c96dc30>,
│   <chip.CertificateAuthority.CertificateAuthority object at 0x74c32c96d720>
]

### Establish CASE and Read OpCreds

To prove that we do indeed have two distinct fabrics and controllers on each fabric, let's go ahead and update the label of each fabric. To do so, you'd need to successfully establish a CASE session through a controller on the respective fabric, and call the 'UpdateLabel' command.

Underneath the covers, each device controller will do operational discovery of the NodeId being read and establish a CASE session before issuing the IM interaction.

In [17]:
devCtrl

In [18]:
await devCtrl.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric1Label"))
await devCtrl.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 4246564830,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04\xffKd\xd3v\xa97c\x87\r\xadXF\xd2?\n\x08\x1f\rZE\x91\x0c_\x1e`"\xfbwZ\x9c\xaf!\x10=\xd4\x03\xf5[}\xb3\x08\xa2\xe5\x04\x7f\x06\x1c\nEa\x02q\xeeK\x91\xda\x1e\xd2\\i\xef\x05#',
│   │   │   │   │   vendorID=65521,
│   │   │   │   │   fabricID=1,
│   │   │   │   │   nodeID=2,
│   │   │   │   │   label='Fabric1Label',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04pKSE\xb6\x07\x04\n\xa5p\xbb\x1f\xa0Z\xb8HS\xaa\xaa\xe9|\xf1\xaf+\x0e\xa6] 7\x8f\x10\xe5\xa6\xb6\x15$k\x08#X\xa5\x8f+\xd3M\x99\x06\xc1\x86\x8e\xda\xd3{&\xaf)\x81\xa1h\xc7$\xc0\xad\x9b',
│   │   │   │   │   vendorID=65521,
│   │   │   │ 

Instantiate a controller on fabric 2 and use it to read out the op creds from that fabric.

In [19]:
fabricAdmin2 = caList[1].adminList[0]
devCtrl2 = fabricAdmin2.NewController()

await devCtrl2.SendCommand(2, 0, Clusters.OperationalCredentials.Commands.UpdateFabricLabel("Fabric2Label"))
await devCtrl2.ReadAttribute(2, [(Clusters.OperationalCredentials.Attributes.Fabrics)], fabricFiltered=False)


{
│   0: {
│   │   <class 'chip.clusters.Objects.OperationalCredentials'>: {
│   │   │   <class 'chip.clusters.Attribute.DataVersion'>: 4246564831,
│   │   │   <class 'chip.clusters.Objects.OperationalCredentials.Attributes.Fabrics'>: [
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04\xffKd\xd3v\xa97c\x87\r\xadXF\xd2?\n\x08\x1f\rZE\x91\x0c_\x1e`"\xfbwZ\x9c\xaf!\x10=\xd4\x03\xf5[}\xb3\x08\xa2\xe5\x04\x7f\x06\x1c\nEa\x02q\xeeK\x91\xda\x1e\xd2\\i\xef\x05#',
│   │   │   │   │   vendorID=65521,
│   │   │   │   │   fabricID=1,
│   │   │   │   │   nodeID=2,
│   │   │   │   │   label='Fabric1Label',
│   │   │   │   │   fabricIndex=1
│   │   │   │   ),
│   │   │   │   FabricDescriptorStruct(
│   │   │   │   │   rootPublicKey=b'\x04pKSE\xb6\x07\x04\n\xa5p\xbb\x1f\xa0Z\xb8HS\xaa\xaa\xe9|\xf1\xaf+\x0e\xa6] 7\x8f\x10\xe5\xa6\xb6\x15$k\x08#X\xa5\x8f+\xd3M\x99\x06\xc1\x86\x8e\xda\xd3{&\xaf)\x81\xa1h\xc7$\xc0\xad\x9b',
│   │   │   │   │   vendorID=65521,
│   │   │   │ 

In [20]:
devCtrl2.Shutdown()